In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
from sklearn.datasets import make_blobs
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("../input/eda-missing-values-outliers/after_eda.csv")
df

,visit_year,visit_week,week_year_visit,restaurant_id,air_store_id,sum_visitors,max_visitors,min_visitors,avg_visitors,count_visitors,...,station_vincenty,station_great_circle,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category
0,2016,1,1_2016,8,air_c225148c0fcc5c72,30,30,30,30,1,...,1.277232,1.274882,34.695124,135.197853,57,30,5,5,5,5
1,2016,1,1_2016,17,air_25d8e5cc57dd87d9,1,1,1,1,1,...,5.319959,5.318636,35.661777,139.704051,444,1,5,5,5,5
2,2016,1,1_2016,18,air_1e23210b584540e7,3,3,3,3,1,...,4.211253,4.202990,35.693840,139.703549,444,3,5,5,5,5
3,2016,1,1_2016,35,air_57ed725a1930a5b9,78,45,33,39,2,...,4.211253,4.202990,35.693840,139.703549,444,78,5,5,5,5
4,2016,1,1_2016,47,air_789466e488705c93,272,68,68,68,4,...,2.256617,2.258974,33.319286,130.508374,127,272,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42276,2017,16,16_2017,825,air_70e9e8cd55879414,76,17,3,10,7,...,6.027513,6.025058,35.659397,139.696440,444,152,5,5,5,5
42277,2017,16,16_2017,826,air_8e4360a64dbd4c50,140,36,12,20,7,...,2.736619,2.742844,34.705362,135.510025,74,280,5,5,5,5
42278,2017,16,16_2017,827,air_f6bfd27e2e174d16,92,31,7,15,6,...,5.319959,5.318636,35.661777,139.704051,444,184,5,5,5,5
42279,2017,16,16_2017,828,air_c3dcaf3aeb18e20e,110,28,1,15,7,...,4.535672,4.541093,35.726232,139.723240,444,220,5,5,5,5


In [3]:
df = df.assign(latitude_longitude = df.restaurant_latitude + df.restaurant_longitude)
df

,visit_year,visit_week,week_year_visit,restaurant_id,air_store_id,sum_visitors,max_visitors,min_visitors,avg_visitors,count_visitors,...,station_great_circle,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude
0,2016,1,1_2016,8,air_c225148c0fcc5c72,30,30,30,30,1,...,1.274882,34.695124,135.197853,57,30,5,5,5,5,169.892977
1,2016,1,1_2016,17,air_25d8e5cc57dd87d9,1,1,1,1,1,...,5.318636,35.661777,139.704051,444,1,5,5,5,5,175.365828
2,2016,1,1_2016,18,air_1e23210b584540e7,3,3,3,3,1,...,4.202990,35.693840,139.703549,444,3,5,5,5,5,175.397390
3,2016,1,1_2016,35,air_57ed725a1930a5b9,78,45,33,39,2,...,4.202990,35.693840,139.703549,444,78,5,5,5,5,175.397390
4,2016,1,1_2016,47,air_789466e488705c93,272,68,68,68,4,...,2.258974,33.319286,130.508374,127,272,1,1,1,1,163.827660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42276,2017,16,16_2017,825,air_70e9e8cd55879414,76,17,3,10,7,...,6.025058,35.659397,139.696440,444,152,5,5,5,5,175.355837
42277,2017,16,16_2017,826,air_8e4360a64dbd4c50,140,36,12,20,7,...,2.742844,34.705362,135.510025,74,280,5,5,5,5,170.215387
42278,2017,16,16_2017,827,air_f6bfd27e2e174d16,92,31,7,15,6,...,5.318636,35.661777,139.704051,444,184,5,5,5,5,175.365828
42279,2017,16,16_2017,828,air_c3dcaf3aeb18e20e,110,28,1,15,7,...,4.541093,35.726232,139.723240,444,220,5,5,5,5,175.449472


In [4]:
new_df = df.drop(['air_store_id','station_name','area','genre','week_year_visit','restaurant_id'], axis=1)
new_df

,visit_year,visit_week,sum_visitors,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,min_reserve_visitors,...,station_great_circle,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude
0,2016,1,30,30,30,30,1,0,0,0,...,1.274882,34.695124,135.197853,57,30,5,5,5,5,169.892977
1,2016,1,1,1,1,1,1,0,0,0,...,5.318636,35.661777,139.704051,444,1,5,5,5,5,175.365828
2,2016,1,3,3,3,3,1,0,0,0,...,4.202990,35.693840,139.703549,444,3,5,5,5,5,175.397390
3,2016,1,78,45,33,39,2,0,0,0,...,4.202990,35.693840,139.703549,444,78,5,5,5,5,175.397390
4,2016,1,272,68,68,68,4,67,41,6,...,2.258974,33.319286,130.508374,127,272,1,1,1,1,163.827660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42276,2017,16,76,17,3,10,7,0,0,0,...,6.025058,35.659397,139.696440,444,152,5,5,5,5,175.355837
42277,2017,16,140,36,12,20,7,0,0,0,...,2.742844,34.705362,135.510025,74,280,5,5,5,5,170.215387
42278,2017,16,92,31,7,15,6,0,0,0,...,5.318636,35.661777,139.704051,444,184,5,5,5,5,175.365828
42279,2017,16,110,28,1,15,7,0,0,0,...,4.541093,35.726232,139.723240,444,220,5,5,5,5,175.449472


In [5]:
new_df.to_csv("./new_df.csv",index = False)
new_df

,visit_year,visit_week,sum_visitors,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,min_reserve_visitors,...,station_great_circle,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude
0,2016,1,30,30,30,30,1,0,0,0,...,1.274882,34.695124,135.197853,57,30,5,5,5,5,169.892977
1,2016,1,1,1,1,1,1,0,0,0,...,5.318636,35.661777,139.704051,444,1,5,5,5,5,175.365828
2,2016,1,3,3,3,3,1,0,0,0,...,4.202990,35.693840,139.703549,444,3,5,5,5,5,175.397390
3,2016,1,78,45,33,39,2,0,0,0,...,4.202990,35.693840,139.703549,444,78,5,5,5,5,175.397390
4,2016,1,272,68,68,68,4,67,41,6,...,2.258974,33.319286,130.508374,127,272,1,1,1,1,163.827660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42276,2017,16,76,17,3,10,7,0,0,0,...,6.025058,35.659397,139.696440,444,152,5,5,5,5,175.355837
42277,2017,16,140,36,12,20,7,0,0,0,...,2.742844,34.705362,135.510025,74,280,5,5,5,5,170.215387
42278,2017,16,92,31,7,15,6,0,0,0,...,5.318636,35.661777,139.704051,444,184,5,5,5,5,175.365828
42279,2017,16,110,28,1,15,7,0,0,0,...,4.541093,35.726232,139.723240,444,220,5,5,5,5,175.449472


In [6]:
new_df1 = new_df.drop(['sum_visitors'], axis=1)
new_df1

,visit_year,visit_week,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,min_reserve_visitors,avg_reserve_visitors,...,station_great_circle,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude
0,2016,1,30,30,30,1,0,0,0,0,...,1.274882,34.695124,135.197853,57,30,5,5,5,5,169.892977
1,2016,1,1,1,1,1,0,0,0,0,...,5.318636,35.661777,139.704051,444,1,5,5,5,5,175.365828
2,2016,1,3,3,3,1,0,0,0,0,...,4.202990,35.693840,139.703549,444,3,5,5,5,5,175.397390
3,2016,1,45,33,39,2,0,0,0,0,...,4.202990,35.693840,139.703549,444,78,5,5,5,5,175.397390
4,2016,1,68,68,68,4,67,41,6,16,...,2.258974,33.319286,130.508374,127,272,1,1,1,1,163.827660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42276,2017,16,17,3,10,7,0,0,0,0,...,6.025058,35.659397,139.696440,444,152,5,5,5,5,175.355837
42277,2017,16,36,12,20,7,0,0,0,0,...,2.742844,34.705362,135.510025,74,280,5,5,5,5,170.215387
42278,2017,16,31,7,15,6,0,0,0,0,...,5.318636,35.661777,139.704051,444,184,5,5,5,5,175.365828
42279,2017,16,28,1,15,7,0,0,0,0,...,4.541093,35.726232,139.723240,444,220,5,5,5,5,175.449472


In [7]:
df_col = new_df1.describe()
#(include ='int64')
df_col.columns

Index(['visit_year', 'visit_week', 'max_visitors', 'min_visitors',
       'avg_visitors', 'count_visitors', 'sum_reserve_visitors',
       'max_reserve_visitors', 'min_reserve_visitors', 'avg_reserve_visitors',
       'count_reserve_visitors', 'holiday_flg_visit', 'holiday_week_visit',
       'number_of_holiday_days_visit', 'area_id', 'genre_id', 'station_id',
       'station_vincenty', 'station_great_circle', 'restaurant_latitude',
       'restaurant_longitude', 'number_of_restaurants_in_area',
       'weekend_visitors', 'sum_diff_days_between_reserve_visit_category',
       'max_diff_days_between_reserve_visit_category',
       'avg_diff_days_between_reserve_visit_category',
       'min_diff_days_between_reserve_visit_category', 'latitude_longitude'],
      dtype='object')

In [8]:
varSel = pd.DataFrame({'Variable': df_col.columns})
varSel

,Variable
0,visit_year
1,visit_week
2,max_visitors
3,min_visitors
4,avg_visitors
5,count_visitors
6,sum_reserve_visitors
7,max_reserve_visitors
8,min_reserve_visitors
9,avg_reserve_visitors


In [9]:
nm = df_col.columns
nm = nm.append(pd.Index(['sum_visitors']))
nm

Index(['visit_year', 'visit_week', 'max_visitors', 'min_visitors',
       'avg_visitors', 'count_visitors', 'sum_reserve_visitors',
       'max_reserve_visitors', 'min_reserve_visitors', 'avg_reserve_visitors',
       'count_reserve_visitors', 'holiday_flg_visit', 'holiday_week_visit',
       'number_of_holiday_days_visit', 'area_id', 'genre_id', 'station_id',
       'station_vincenty', 'station_great_circle', 'restaurant_latitude',
       'restaurant_longitude', 'number_of_restaurants_in_area',
       'weekend_visitors', 'sum_diff_days_between_reserve_visit_category',
       'max_diff_days_between_reserve_visit_category',
       'avg_diff_days_between_reserve_visit_category',
       'min_diff_days_between_reserve_visit_category', 'latitude_longitude',
       'sum_visitors'],
      dtype='object')

In [10]:
df2 = df[nm].copy()
df2.head()

,visit_year,visit_week,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,min_reserve_visitors,avg_reserve_visitors,...,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude,sum_visitors
0,2016,1,30,30,30,1,0,0,0,0,...,34.695124,135.197853,57,30,5,5,5,5,169.892977,30
1,2016,1,1,1,1,1,0,0,0,0,...,35.661777,139.704051,444,1,5,5,5,5,175.365828,1
2,2016,1,3,3,3,1,0,0,0,0,...,35.693840,139.703549,444,3,5,5,5,5,175.397390,3
3,2016,1,45,33,39,2,0,0,0,0,...,35.693840,139.703549,444,78,5,5,5,5,175.397390,78
4,2016,1,68,68,68,4,67,41,6,16,...,33.319286,130.508374,127,272,1,1,1,1,163.827660,272


In [11]:
X = df2.iloc[:,:-1]
X.head()

,visit_year,visit_week,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,min_reserve_visitors,avg_reserve_visitors,...,station_great_circle,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude
0,2016,1,30,30,30,1,0,0,0,0,...,1.274882,34.695124,135.197853,57,30,5,5,5,5,169.892977
1,2016,1,1,1,1,1,0,0,0,0,...,5.318636,35.661777,139.704051,444,1,5,5,5,5,175.365828
2,2016,1,3,3,3,1,0,0,0,0,...,4.202990,35.693840,139.703549,444,3,5,5,5,5,175.397390
3,2016,1,45,33,39,2,0,0,0,0,...,4.202990,35.693840,139.703549,444,78,5,5,5,5,175.397390
4,2016,1,68,68,68,4,67,41,6,16,...,2.258974,33.319286,130.508374,127,272,1,1,1,1,163.827660


In [12]:
y = df2.iloc[:,-1:]
y.head()

,sum_visitors
0,30
1,1
2,3
3,78
4,272


### Variable Selection using LASSO (L1 penalization)

In [13]:
lassomod = Lasso(alpha=0.01).fit(X, y.values.ravel())

In [14]:
model = SelectFromModel(lassomod, prefit=True)
model.get_support()

array([ True,  True,  True,  True,  True,  True,  True, False, False,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False, False, False,  True,  True,  True, False, False,  True,
        True])

In [15]:
varSel['Lasso'] = model.get_support()
#.astype('int64')
varSel

,Variable,Lasso
0,visit_year,True
1,visit_week,True
2,max_visitors,True
3,min_visitors,True
4,avg_visitors,True
5,count_visitors,True
6,sum_reserve_visitors,True
7,max_reserve_visitors,False
8,min_reserve_visitors,False
9,avg_reserve_visitors,True


### Variable Selection using XGboost

In [16]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10).fit(X,y.values.ravel())

[15:22:40] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.


In [17]:
model = SelectFromModel(xg_reg, prefit=True)
model.get_support()

array([False, False,  True, False,  True,  True,  True,  True, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False,  True, False,
       False])

In [18]:
varSel['XGboost'] = model.get_support()
#.astype('int64')
varSel

,Variable,Lasso,XGboost
0,visit_year,True,False
1,visit_week,True,False
2,max_visitors,True,True
3,min_visitors,True,False
4,avg_visitors,True,True
5,count_visitors,True,True
6,sum_reserve_visitors,True,True
7,max_reserve_visitors,False,True
8,min_reserve_visitors,False,False
9,avg_reserve_visitors,True,False


### Summarization and Selection of Variables 


In [19]:
varSel['Sum'] = np.sum(varSel,axis=1)
varSel

,Variable,Lasso,XGboost,Sum
0,visit_year,True,False,1
1,visit_week,True,False,1
2,max_visitors,True,True,2
3,min_visitors,True,False,1
4,avg_visitors,True,True,2
5,count_visitors,True,True,2
6,sum_reserve_visitors,True,True,2
7,max_reserve_visitors,False,True,1
8,min_reserve_visitors,False,False,0
9,avg_reserve_visitors,True,False,1


In [20]:
varSel.groupby('Sum')['Variable'].count()

Sum
0     6
1    16
2     6
Name: Variable, dtype: int64

In [21]:
# We can now decide a threshold for selecting our variables!

In [22]:
varSel[varSel['Sum']>=1]

,Variable,Lasso,XGboost,Sum
0,visit_year,True,False,1
1,visit_week,True,False,1
2,max_visitors,True,True,2
3,min_visitors,True,False,1
4,avg_visitors,True,True,2
5,count_visitors,True,True,2
6,sum_reserve_visitors,True,True,2
7,max_reserve_visitors,False,True,1
9,avg_reserve_visitors,True,False,1
10,count_reserve_visitors,True,True,2


In [23]:
variables = varSel[varSel['Sum']>=1].iloc[:, [0]]
variables

,Variable
0,visit_year
1,visit_week
2,max_visitors
3,min_visitors
4,avg_visitors
5,count_visitors
6,sum_reserve_visitors
7,max_reserve_visitors
9,avg_reserve_visitors
10,count_reserve_visitors


In [24]:
variables_to_columns = variables.transpose().iloc[:].head()
df_variables_to_columns = pd.DataFrame(variables_to_columns)
sum_visitors = ['sum_visitors']
df_variables_to_columns['sum_visitors'] = sum_visitors
df_variables_to_columns.to_csv('first_variables_data.csv',index = False)
df_variables_to_columns

,0,1,2,3,4,5,6,7,9,10,...,15,16,17,21,22,23,25,26,27,sum_visitors
Variable,visit_year,visit_week,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,avg_reserve_visitors,count_reserve_visitors,...,genre_id,station_id,station_vincenty,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude,sum_visitors


In [25]:
top_columns = pd.read_csv("./first_variables_data.csv",header = 1).columns
top_columns

Index(['visit_year', 'visit_week', 'max_visitors', 'min_visitors',
       'avg_visitors', 'count_visitors', 'sum_reserve_visitors',
       'max_reserve_visitors', 'avg_reserve_visitors',
       'count_reserve_visitors', 'holiday_flg_visit',
       'number_of_holiday_days_visit', 'area_id', 'genre_id', 'station_id',
       'station_vincenty', 'number_of_restaurants_in_area', 'weekend_visitors',
       'sum_diff_days_between_reserve_visit_category',
       'avg_diff_days_between_reserve_visit_category',
       'min_diff_days_between_reserve_visit_category', 'latitude_longitude',
       'sum_visitors'],
      dtype='object')

In [26]:
df_After_Feature_Selection = pd.DataFrame(new_df[top_columns])
df_After_Feature_Selection.to_csv("./First_Feature_Selection.csv",index = False)
df_After_Feature_Selection

,visit_year,visit_week,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,avg_reserve_visitors,count_reserve_visitors,...,genre_id,station_id,station_vincenty,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude,sum_visitors
0,2016,1,30,30,30,1,0,0,0,0,...,4,15,1.277232,57,30,5,5,5,169.892977,30
1,2016,1,1,1,1,1,0,0,0,0,...,1,37,5.319959,444,1,5,5,5,175.365828,1
2,2016,1,3,3,3,1,0,0,0,0,...,9,37,4.211253,444,3,5,5,5,175.397390,3
3,2016,1,45,33,39,2,0,0,0,0,...,4,37,4.211253,444,78,5,5,5,175.397390,78
4,2016,1,68,68,68,4,67,41,16,4,...,13,4,2.256617,127,272,1,1,1,163.827660,272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42276,2017,16,17,3,10,7,0,0,0,0,...,6,37,6.027513,444,152,5,5,5,175.355837,76
42277,2017,16,36,12,20,7,0,0,0,0,...,4,25,2.736619,74,280,5,5,5,170.215387,140
42278,2017,16,31,7,15,6,0,0,0,0,...,13,37,5.319959,444,184,5,5,5,175.365828,92
42279,2017,16,28,1,15,7,0,0,0,0,...,6,37,4.535672,444,220,5,5,5,175.449472,110
